In [2]:
import pandas as pd
import numpy as np

In [3]:
# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP23&subject=CS" # get SP 22 CS classes
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())


In [4]:
classes = data_json['data']['classes']

In [5]:
url2 = 'https://classes.cornell.edu/api/2.0/config/subjects.json?roster=SP23'
response2 = urlopen(url2)
subjects_json = json.loads(response2.read())

In [6]:
subject_list = []
for i in subjects_json['data']['subjects']:
    subject_list.append(i['value'])


In [7]:
df = []
missedClasses = 0
cntr = 0
for subject in subject_list: 
    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP23&subject={subject}"
    response = urlopen(url)

    data_json = json.loads(response.read())

    classes = data_json['data']['classes']

    for class_ in classes:
        try:        
            obj = class_['enrollGroups'][0]['classSections'][0]['meetings'][0]
            try:
                obj2 = obj['instructors'][0]
                prof = obj2['firstName'].strip() + ' ' + obj2['lastName'].strip()
            except:
                prof = ""

            startTime = obj['timeStart']
            endTime = obj['timeEnd']
            location = obj['facilityDescr']
            #x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], obj2['firstName'].strip() + ' ' + obj2['lastName'].strip(), startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]
            x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], prof, startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]

            df.append(x)
        except Exception as e:
            print(cntr)
            print(class_)
            print('\n')
            print(e)
            print('\n')
            missedClasses += 1
        cntr +=1
print(missedClasses)

631
{'strm': 2839, 'crseId': 365420, 'crseOfferNbr': 1, 'subject': 'ASRC', 'catalogNbr': '3508', 'titleShort': 'Afr Am Lit: 1930s- Present', 'titleLong': 'African American Literature: 1930s - Present', 'enrollGroups': [{'classSections': [{'ssrComponent': 'LEC', 'ssrComponentLong': 'Lecture', 'section': '001', 'classNbr': 18924, 'meetings': [], 'notes': [], 'campus': 'MAIN', 'campusDescr': 'Ithaca', 'location': 'ITH', 'locationDescr': 'Ithaca, NY (Main Campus)', 'startDt': '01/23/2023', 'endDt': '05/20/2023', 'addConsent': 'N', 'addConsentDescr': 'No Special Consent Required', 'isComponentGraded': True, 'instructionMode': 'P', 'instrModeDescrshort': 'In Person', 'instrModeDescr': 'In Person', 'topicDescription': '', 'exploreCriteriaIds': [], 'materials': []}], 'unitsMinimum': 4, 'unitsMaximum': 4, 'componentsOptional': [], 'componentsRequired': ['LEC'], 'gradingBasis': 'GRD', 'gradingBasisShort': 'Graded', 'gradingBasisLong': 'Letter grades only', 'simpleCombinations': [{'subject': 'ENG

In [8]:
df1 = pd.DataFrame(np.array(df))
df1.shape
#Sp22: 4544, FA22: 4162

(4519, 10)

In [9]:
df1 = df1.rename(columns={0:"Dept", 1:"Number", 2:"Course_Name", 3:"Professor", 4:"Start_Time", 5:"End_Time", 6:"Location",7:"Description",8:"Pre/coreqs",9:"Offered"})

In [11]:
median_grades = pd.read_csv("Cleaned_Medians.csv")

In [12]:
median_grades

,Unnamed: 0,Dept + Number,Professor,Median Grade,Semester (Ex: SP21),# of Students,Notes,Semester
0,1,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640",0.0
1,289,ENGL 1158 (FWS: Asian American Food Writing),Rhee,A,FA21,17,NaN,0.0
2,585,PLBIO 2300,Nixon,A,FA21,13,NaN,0.0
3,287,ENGL 1105,Various (FWS),A+,FA21,49,NaN,0.0
4,286,EDUC 2610,Steinhall,A,FA21,86,NaN,0.0
...,...,...,...,...,...,...,...,...
519,227,DEA 3510,Hedge,B,FA18,14,NaN,6.0
520,175,COMM 2200,Humphreys,A-,FA18,95,Cookie Cutter Essays,6.0
521,140,CEE 4510,Richardson,A-,FA17,44,NaN,8.0
522,171,COML 1109,Various (FWS),A-,FA17,NaN,NaN,8.0


In [38]:
median_grades_v2 = pd.read_csv("Cornell_Median_Grades.csv", names = ["Class_Name", "professor", "median", "semester", "size","notes"])
median_grades_v2

,Class_Name,professor,median,semester,size,notes
0,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640"
1,AAS 2130,Moghimi,A,FA21,NaN,NaN
2,AEM 2010,Moghimi,A+,SP22,145,NaN
3,AEM 2100,Van Es,A,FA21,252,NaN
4,AEM 2210,Riley,A,SP21,NaN,NaN
...,...,...,...,...,...,...
794,STSCI 2100,Packard/Karns,B+,FA21,234,crosslisted with ILRST 2100
795,STSCI 2150,Smith,B+,SP22,211,NaN
796,Dept + Number,Professor,Median Grade,SP22,# of Students,Notes
797,GOVT 3867,Ward,NaN,FA20,NaN,NaN


In [39]:
#UPDATE THIS IF NEEDED WHEN RE RUN
semesterdict = {"FA22":-2,"SP22":-1, "FA21":0, 'SP21':1, 'FA20':2, 'SP20':3, 'FA19':4,'SP19':5,'FA18':6,'SP18':7,'FA17':8,'SP17':9,'FA16':10}

In [40]:
median_grades_v2.head()

,Class_Name,professor,median,semester,size,notes
0,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640"
1,AAS 2130,Moghimi,A,FA21,NaN,NaN
2,AEM 2010,Moghimi,A+,SP22,145,NaN
3,AEM 2100,Van Es,A,FA21,252,NaN
4,AEM 2210,Riley,A,SP21,NaN,NaN


In [41]:
#Data cleaning for medians
median_grades_v2['SemesterN'] = median_grades_v2['semester']
median_grades_v2 = median_grades_v2.replace({"SemesterN": semesterdict})
median_grades_v2['SemesterN'] = median_grades_v2['SemesterN'].astype(int, errors = "ignore")
median_grades_v2 = median_grades_v2.sort_values(by="SemesterN")
median_grades_v2 = median_grades_v2.drop_duplicates(subset="Class_Name", keep="first")

In [43]:
#median_grades_v2 = median_grades_v2[['Dept + Number', 'Median Grade', 'Semester (Ex: SP21)', 'Semester']]
median_grades_v2.head()

,Class_Name,professor,median,semester,size,notes,SemesterN
495,ILRID 1520,Bohns,A-,FA22,NaN,1.5 Credit Class,-2.0
769,PAM 2000,Tripp,B+,SP22,87,NaN,-1.0
457,PAM 3764,Manne,A-,SP22,70,NaN,-1.0
229,PSYCH 1102,Nichols,A,SP22,271,"crosslisted with COGST 1101, LING 1170, PHIL 1...",-1.0
119,ENGRI 1100,McMahon,A,SP22,26,crosslisted with AEP 1100,-1.0


In [44]:
df1

,Dept,Number,Course_Name,Professor,Start_Time,End_Time,Location,Description,Pre/coreqs,Offered
0,AAS,1100,Introduction to Asian American Studies,Viranjini Munasinghe,02:55PM,04:10PM,To Be Assigned,This interdisciplinary course offers an introd...,,Spring.
1,AAS,2620,Introduction to Asian American Literature,Juliana Hu Pegues,11:25AM,12:40PM,To Be Assigned,This course will introduce both a variety of w...,,Spring.
2,AAS,3312,Afro-Asia: Futurism and Feminism,Tao Goffe,10:10AM,11:25AM,To Be Assigned,This course explores cultural representations ...,,Spring.
3,AAS,3378,Korean American Literature,Ivanna Yi,11:25AM,12:40PM,To Be Assigned,The rapidly growing literature of the Korean d...,,Spring.
4,AAS,4950,Independent Study,Viranjini Munasinghe,,,None,Independent reading course in topics not cover...,,"Fall, Spring."
...,...,...,...,...,...,...,...,...,...,...
4514,YORUB,2111,Intermediate Yoruba II,Adeolu Ademoyo,09:05AM,09:55AM,To Be Assigned,Intermediate Yoruba II is a follow-up to Inter...,Prerequisite: YORUB 1108 and YORUB 1109.,Spring.
4515,YORUB,3111,Advanced Yoruba II,Adeolu Ademoyo,07:30PM,09:25PM,To Be Assigned,This course will help students expand their un...,Prerequisite: YORUB 3110.,Spring.
4516,ZULU,1116,Elementary Zulu II,Angelika Kraemer,11:35AM,12:25PM,To Be Assigned,Development of communication skills through di...,,Spring.
4517,ZULU,2117,Intermediate Zulu II,Angelika Kraemer,09:25AM,10:12AM,To Be Assigned,Students read longer texts from popular media ...,,Spring.


In [45]:
df1["Dept + Number"] = df1["Dept"] + ' ' + df1["Number"]

In [46]:
df1['Dept + Number'].value_counts()

LATA 6000     1
LAW 7762      1
ENGRG 4815    1
HIST 2354     1
ARTH 2221     1
             ..
GOVT 6223     1
PE 1333       1
ALS 6016      1
NRE 5030      1
HD 4940       1
Name: Dept + Number, Length: 4519, dtype: int64

In [47]:
df2 = df1.merge(median_grades_v2, left_on='Dept + Number', right_on = "Class_Name", how="outer")

In [49]:
df2 = df2.dropna(subset=['Dept'])

In [53]:
df2[df2['Dept'] == 'INFO'].head(20)

,Dept,Number,Course_Name,Professor,Start_Time,End_Time,Location,Description,Pre/coreqs,Offered,Dept + Number,Class_Name,professor,median,semester,size,notes,SemesterN
2379,INFO,1300,Introductory Design and Programming for the Web,Royal Westwater,11:25AM,12:40PM,To Be Assigned,The World Wide Web is both a technology and a ...,,"Fall, Spring.",INFO 1300,INFO 1300,Harms,A,FA21,264,NaN,0.0
2380,INFO,1380,Data Science for All,Sumanta Basu,09:40AM,10:55AM,To Be Assigned,This course provides an introduction to data s...,,Spring.,INFO 1380,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2381,INFO,1998,Freshmen Team Projects,Walker White,07:30PM,08:20PM,To Be Assigned,Independent work in information science as par...,,"Fall, Spring.",INFO 1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2382,INFO,2300,Intermediate Design and Programming for the Web,Kyle Harms,09:05AM,09:55AM,To Be Assigned,Web programming requires the cooperation of tw...,Prerequisite: INFO 1300.,Spring.,INFO 2300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2383,INFO,2770,Excursions in Computational Sustainability,Carla Gomes,01:00PM,02:15PM,To Be Assigned,"Balancing environmental, economic, and societa...",,Spring.,INFO 2770,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2384,INFO,2921,Inventing an Information Society,Gili Vidan,09:40AM,10:55AM,To Be Assigned,Provides an introduction to the role computing...,,Spring.,INFO 2921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2385,INFO,2950,Introduction to Data Science,Benjamin Soltoff,09:40AM,10:55AM,To Be Assigned,Teaches basic mathematical methods for informa...,"Prerequisite: MATH 1710 or equivalent, CS 1110...",Fall.,INFO 2950,INFO 2950,Wilkens,A-,SP22,217,NaN,-1.0
2386,INFO,3140,Computational Psychology,Shimon Edelman,09:40AM,10:55AM,To Be Assigned,This course states and motivates the observati...,Prerequisite: one course each in psychology an...,Spring.,INFO 3140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2387,INFO,3152,Introduction to Computer Game Design,Walker White,12:25PM,01:15PM,To Be Assigned,A project-based course in which programmers an...,"Prerequisite for programmers: CS 2110, or perm...",Spring.,INFO 3152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2388,INFO,3200,New Media and Society,Chao Yu,09:40AM,10:55AM,To Be Assigned,Media and culture are undergoing a series of t...,,"Spring, Summer.",INFO 3200,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df2.to_csv("Classes_With_Medians_SP23.csv")